# Sequence Modeling and RNNs

Sequential data has to be modeled differently, since the assumption that the data generated is i.i.d is violated, along with the assumptions that the data is drawn from the same underlying distribution. Modeling sequences is quite important since it plays into time series modeling, NLP, or DNA sequencing. Recurrent Neural Networks (RNNs) are well suited to this task.

## Sequence Modeling

Sequences will be denoted $\langle x^{(1)}, x^{(2)}, \dots, x^{(T)} \rangle$. There are many architectures that can model sequences, but each architecture corresponds to different types of usages.

1) Many-to-one: The input data is a sequence, but the output is a fixed-size tensor, and not a sequence. This plays a role in sentiment analysis or image generation.

2) One-to-many: Input data is not a sequence, but in some standard format, whereas the output is a sequence. This has applications in image captioning and similar tasks.

3) Many-to-many: Both input and output data are sequences. There are many types of use cases for this, but can be broken down into synchronized modeling (such as live captioning of a video by frames), or delayed modeling, such as language translation (where an entire sentence must be read before translation can occur).

## Structure of an RNN

### Dataflow
A regular feed forward neural network has a dataflow in one direction, from input to the first hidden layer (if applicable), and a one hidden layer to the next, until the last one feeds into the output. A RNN has a similar structure, but the hidden layers feed into themselves in a non-sequential manner. More particularly, a hidden layer receives inputs from the prior layer, but additionally received inputs from its last state. More particularly, hidden layers at step/time $t$ receive inputs from their state at time $t-1$ as well as their input from the preceeding layer at time $t$. Thus, there are weights that map from one layer to the next, as well as from the layer at a prior state to the current state: $$z^{(t)}_h = W_{xh} x^{(t)} + W_{hh} h^{(t- 1)} + b_h; \qquad h^{(t)} = \sigma_h(z_h^{(t)}).$$

When it comes to recurrence structures, this is just one formulation. There are alternatives that use recurrence with respect to the output. What was discussed above was the hidden-to-hidden recurrence structure. There are also output recurrect structures, broken into 2 categories:

1) Output-to-hidden recurrence: Input from the prior time step comes from the output layer instead of the hidden layers to themselves. Effectively, we replace the $W_{hh} h^{(t- 1)}$ term in  $z^{(t)}$ above with $W_{oh} o^{(t- 1)}$, with $o^{(t)}$ denoting an output at time step $t$.

2) Output-to-output recurrence: This architecture follows a structure similar to a regular NN until the output layer. The output layer at time $t$ receives inputs from the last hidden layer as well as the output at time $t-1$.

### Loss
The loss is the sum of the loss at all time steps: $$L \sum_{t = 1}^T L^{(t)}.$$ The chain rule to derivate outputs is similar to a regular NN, but requires chaining across another dimension/index: $$\frac{\partial L^{(t)}}{\partial W_{hh}} = \frac{\partial L^{(t)}}{\partial o^{(t)}} \times \frac{\partial o^{(t)}}{\partial h^{(t)}} \times \left( \sum_{\tau = 1}^t \frac{\partial h^{(t)}}{\partial h^{(\tau)}} \times  \frac{\partial h^{(\tau)}}{\partial W_{hh}} \right), \qquad \frac{\partial h^{(t)}}{\partial h^{(\tau)}} = \prod_{j = \tau + 1}^t \frac{\partial h^{(j)}}{\partial h^{(j - 1)}},$$ where $o^{(t)}$ denotes the output at time step $t$.

In [1]:
# Implement an RNN layer as an example
import torch
import torch.nn as nn

torch.manual_seed(42)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

# Weights from input as well as weights from previous step
w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0

# Bias correponding to input and bias corresponding to layer at prior step
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0


print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

W_xh shape: torch.Size([2, 5])
W_hh shape: torch.Size([2, 2])
b_xh shape: torch.Size([2])
b_hh shape: torch.Size([2])


In [2]:
# Analyse the forward pass through the layer
import numpy as np

# Create manual input
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

# Manually compute the output
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))
manual_out = []

for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))

    print(f'Time step {t} -')
    print(f'Input: {xt.numpy()}')

    h_t = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print(f'Hidden: {h_t.detach().numpy()}')

    # Get the previous state, or replace with a 0 tensor if it is the first step
    if manual_out:
        prev_h = manual_out[t - 1]
    else:
        prev_h = torch.zeros((h_t.shape))
    
    o_t = h_t + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    # Default activation with RNNs is tanh
    o_t = torch.tanh(o_t)
    manual_out.append(o_t)

    print(f'Manual Output: {o_t.detach().numpy()}')
    print(f'Automatic RNN Output: {output[:, t].detach().numpy()} \n')

Time step 0 -
Input: [[1. 1. 1. 1. 1.]]
Hidden: [[1.7974439  0.21845508]]
Manual Output: [[0.981705   0.31219152]]
Automatic RNN Output: [[0.981705   0.31219152]] 

Time step 1 -
Input: [[2. 2. 2. 2. 2.]]
Hidden: [[3.2539294 0.5367473]]
Manual Output: [[0.9997238  0.82871497]]
Automatic RNN Output: [[0.9997238  0.82871497]] 

Time step 2 -
Input: [[3. 3. 3. 3. 3.]]
Hidden: [[4.710415   0.85503954]]
Manual Output: [[0.9999872 0.915613 ]]
Automatic RNN Output: [[0.9999872 0.915613 ]] 



## Long-range Interactions: Vanishing and Exploding Gradients

In order to compute the gradient of the loss with respect to hidden layers, the chain rule necessitates the computation of the derivative of a hidden layer $k = t - \tau$ steps ago. This can be computed as the product across single time steps, using $k$ chained multiplicative factors. If these updates $\partial h^{(j)}/ \partial{h^{(j - 1)}}$ involve a weight $w$ such that $|w| > 1$/$|w$ < 1$, we see an issue with exploding/vanishing gradients, implying a lack of numerical stability across large sequences. To prevent this, we may require $|w| = 1$, normalizing the weights, which turns out to be the naive solution, and adds significant computational expense whilst slowing convergence. In practice, three major ways of dealing with this are:

1) Gradient clipping: Specify a cut-off or threshold value for gradients, and assign that ceiling to any gradients that exceed this.

2) Truncate backpropagation through time (TBPTT): Limit the number of time steps that the signal can backpropagate after each pass, to the $T_{trunc}$ most recent time steps.

3) Long short-term memory (LSTM): The LSTM mechanism overcomes the issue of vanishing gradients along whilst allowing the gradient to flow backward through time as far as needed to update weights, unlike the other 2 methods. This involves replacing a hidden layer with a memory cell.

### Memory Cells

In each memory cell, there is a recurrent edge that has weights with $|w| = 1$. The values associated with the recurrent edge comprise the cell state. Let the cell state at time $t$ be denoted by $C^{(t)}$. The cell state is modified from $t - 1$ to $t$ by the input and the hidden layer's prior value via 3 gates and a candidate value.

1) forget gate: The forget gate, denoted $f_t$ allows the memory cell to modify the prior cell state without indefinite growth, filtering some parts of the information out. The forget gate is computed as $$f_t = \sigma \left( W_{xf}x^{(t)} + W_{hf}h^{(t - 1)} + b_f \right),$$ where $\sigma$ denotes the sigmoid function. The forget gate will be multiplied, element-wise, with the prior cell state, to determine what fraction of each element is "forgotten" as $C^{(t-1)} \odot f_t$, where $\odot$ is the Hadamard product.

2) input gate: The input gate, $i_t$, controls the amount that the new candidate value can modify the old cell state, through a similar Hadamard product. The candidate value, $\tilde{C}^{(t)}$ is also computed at this step. $$i_t = \sigma \left( W_{xi}x^{(t)} + W_{hi}h^{(t - 1)} + b_i \right), \qquad \tilde{C}^{(t)} = \tanh \left( W_{xc}x^{(t)} + W_{hc}h^{(t - 1)} + b_c \right).$$

3) output gate: This gate, $o_t$ detemines the element wise strength of the update to the hidden layer through a Hadamard product with it. The output gate follows a near-identical computation to that of the other 2 gates: $$o_t = \sigma \left( W_{xo}x^{(t)} + W_{ho}h^{(t - 1)} + b_o \right).$$


Putting them all together, the update rules are: $$ C^{(t)} = \left(C^{(t-1)} \odot f_t \right) + \left(\tilde{C}^{(t)} \odot i_t \right), \qquad h^{(t)} = o_t \odot \tanh(C^{(t)}),$$ where the sum happens element-wise. With these fractional memory updates, the problem with exploding gradients is ameliorated.

## Bi-directional LSTMs

In a regular LSTM, the update is done on the backward pass but where data flows sequentially increasing in layer depth and chronologically ordered in time steps. However, in a bi-directional LSTM, cells also pass information in the reverse direction in time step. This can also be implemented in PyTorch.

A uni-directional forward LTSM preserves information from the past (in terms of time steps/chronological ordering). A backward LTSM (not to be confused with a backward pass to update the weights) takes in information from later time steps and feeds it backwards. This provides more context to the interpretation of past signals. For example, the sentence "The bow drew tremendous appreciation from his customer, once it was strung." would not provide enough context to interpret the word bow if truncated before the word "strung". A bi-directional LSTM allows for this context to be preserved during interpretation.

Below, we construct an example of a bi-directional LSTM. The uni-directional version can be found in the IMDb senitment analysis with RNNs, which will be modified 

In [3]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) -> None:
        super().__init__()

        # Start with an embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim=embed_dim, padding_idx=0)
        # Implement the RNN layer - uses an LSTM here
        # Modified to have bidirectional option to True (default=false)
        self.rnn = nn.LSTM(input_size=embed_dim, hidden_size=rnn_hidden_size, bidirectional=True)

        # On to regular fully connected layers
        # We need to double the number of features coming into the fully connected layer
        # since the
        self.fc1 = nn.Linear(in_features=rnn_hidden_size * 2, out_features=fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=fc_hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, text, lengths):
        out = self.embedding(text)
        # pad sequences to be of the same length, leaving them unsorted
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)

        out, (hidden, cell) = self.rnn(out)
        # Take the last time step output
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)

        return out

model = RNN(10000, 20, 64, 64)
print(model)

RNN(
  (embedding): Embedding(10000, 20, padding_idx=0)
  (rnn): LSTM(20, 64, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
